
- Most Quantum Cloud Services (QCS) users will want to use the Rigetti-hosted JupyterLab IDE to get a personal development environment with everything they need to get started. In some cases, users may choose to Install Locally. However, support for running from a local environment is limited.
- Installing locally is most useful for development using a Quantum Virtual Machine (QVM). Running programs against a QPU requires a reservation as well as network access to the QPU (available via your provisioned JupyterLab IDE).
- To make full use of pyQuil, you’ll want to have both the Quantum Virtual Machine (QVM) and the Quil Compiler (quilc) installed


## Version Information

<table align="left">
  <thead>
    <tr>
      <th>Component</th>
      <th>Version</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>python</td>
      <td>3.11.5</td>
    </tr>
    <tr>
      <td>pyquil</td>
      <td>4.10.0</td>
    </tr>
  </tbody>
</table>

</table>


In [1]:
from pyquil import Program, get_qc
from pyquil.gates import *
from pyquil.quilbase import Declare

In [2]:
p = Program(
    Declare("ro", "BIT", 2),
    H(0),
    CNOT(0, 1),
    MEASURE(0, ("ro", 0)),
    MEASURE(1, ("ro", 1)),
).wrap_in_numshots_loop(10)

After install and activate docker on the machine, run the following commands:

- `docker run --rm -it -p 5555:5555 rigetti/quilc -P -S`

- `docker run --rm -it -p 5000:5000 rigetti/qvm -S`

In [8]:
# # run the program on a QVM
# qc = get_qc('9q-square-qvm')
# result = qc.run(qc.compile(p)).get_register_map().get("ro")
# print(result[0])
# print(result[1])

In [9]:
# Assuming `p` is your quantum program
qc = get_qc('9q-square-qvm')
executable = qc.compile(p)
result = qc.run(executable)

# Access the results
measurement_results = result.readout_data.get('ro')
print(measurement_results[0])
print(measurement_results[1])


[0 0]
[1 1]
